In [ ]:
import cv2
import numpy as np
import pylab

# Kmeans 
def kmeans_color_quantization(image, clusters=8, rounds=1):
    h, w = image.shape[:2]
    samples = np.zeros([h*w,3], dtype=np.float32)
    count = 0

    for x in range(h):
        for y in range(w):
            samples[count] = image[x][y]
            count += 1

    compactness, labels, centers = cv2.kmeans(samples,
            clusters,
            None,
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10000, 0.0001), 
            rounds,
            cv2.KMEANS_RANDOM_CENTERS)

    centers = np.uint8(centers)
    res = centers[labels.flatten()]
    return res.reshape((image.shape))

# Load image
image = cv2.imread('./samples/Microscope Sample Feb 25 Sawdust V2 (2).jpg')
original = image.copy()

# Perform kmeans color segmentation, grayscale, Otsu's threshold
kmeans = kmeans_color_quantization(image, clusters=2)
gray = cv2.cvtColor(kmeans, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Find contours, remove tiny specs using contour area filtering, gather points
points_list = []
size_list = []
cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
AREA_THRESHOLD = 2
for c in cnts:
    area = cv2.contourArea(c)
    if area < AREA_THRESHOLD:
        cv2.drawContours(thresh, [c], -1, 0, -1)
    else:
        (x, y), radius = cv2.minEnclosingCircle(c)
        points_list.append((int(x), int(y)))
        size_list.append(area)

# Apply mask onto original image
result = cv2.bitwise_and(original, original, mask=thresh)
result[thresh==255] = (0,0,255)

# Overlay on original
original[thresh==255] = (0,0,255)

print("Number of particles: {}".format(len(points_list)))
print("Average particle size: {:.3f}".format(sum(size_list)/len(size_list)))


# Display
#cv2.imshow('kmeans', kmeans)
cv2.imshow('original', original)
#cv2.imshow('thresh', thresh)
#cv2.imshow('result', result)
cv2.waitKey()

Number of particles: 8
Average particle size: 98812.500


In [4]:
import cv2
import numpy as np
import os
import glob

def kmeans_color_quantization(image, clusters=8, rounds=1):
    h, w = image.shape[:2]
    samples = np.zeros([h*w,3], dtype=np.float32)
    for x in range(h):
        for y in range(w):
            samples[x*w + y] = image[x][y]

    compactness, labels, centers = cv2.kmeans(samples, clusters, None, (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10000, 0.0001), rounds, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    res = centers[labels.flatten()]
    return res.reshape((image.shape))

def process_images_from_folder(folder_path, results_folder):
    if not os.path.exists(results_folder):
        os.makedirs(results_folder)
        
    for image_path in glob.glob(os.path.join(folder_path, '*')):
        image = cv2.imread(image_path)
        original = image.copy()
        kmeans = kmeans_color_quantization(image, clusters=2)
        gray = cv2.cvtColor(kmeans, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
        AREA_THRESHOLD = 2
        for c in cnts:
            area = cv2.contourArea(c)
            if area < AREA_THRESHOLD:
                cv2.drawContours(thresh, [c], -1, 0, -1)
        result = cv2.bitwise_and(original, original, mask=thresh)
        result[thresh==255] = (0,0,200)
        original[thresh==255] = (0,0,200)
        # Save the processed image
        base_name = os.path.basename(image_path)
        save_path = os.path.join(results_folder, base_name)
        cv2.imwrite(save_path, original)

# Set your folder path and results folder name
folder_path = './samples'
results_folder = 'results_kmeans'
process_images_from_folder(folder_path, results_folder)
